# Shallow Water Modeling

In this notebook, we're going to run an MPI implementation of the shallow water model on a Banyan computing session with a number of workers running in parallel. Banyan automatically sets up cloud computing resources in your own Virtual Private Cloud, sets up MPI and Julia, manages your software environment, lets you run with any number of workers, and scales down when you're finished. All your estimated costs, running clusters, sessions, and logs can be viewed on the [Banyan dashboard](https://www.banyancomputing.com/dashboard).

To run this notebook, you will need to have set up an account with Banyan. Check
out the documentation [here](https://www.banyancomputing.com/getting-started/)
for step by step instructions.

## Configuring

To run this example, please ensure that you have set up your Banyan account.

Run the first cell below to import `Banyan.jl`.
To configure your AWS credentials, run the second cell below and provide your
AWS credentials when prompted. Banyan does not save your AWS credentials, but
they are needed so that you can run your computation in your AWS account.
Finally, run the third cell below to set your Banyan credentials and configure
Banyan.

You must pass your User ID and API Key to the `configure` function in order
to authenticate. You can find this information on the Account page of the
Banyan Dashboard. After running this cell, your credentials will be saved
in `$HOME/.banyan/banyanconfig.toml` and will be read from that file in the
future. This means that you only need to run this cell once.

In [ ]:
# Import packages

using Banyan

In [ ]:
# Run this cell to configure the AWS CLI. When prompted, specify your AWS
# credentials for the AWS account that you connected with Banyan. If you have
# already configured the AWS CLI with the credentials for the account you have
# configured with your Banyan account, you can skip this step.

print("Enter AWS_ACCESS_KEY_ID: \n"); sleep(1)
ENV["AWS_ACCESS_KEY_ID"] = readline()
print("Enter AWS_SECRET_ACCESS_KEY: \n"); sleep(1)
ENV["AWS_SECRET_ACCESS_KEY"] = readline()
print("Enter AWS_DEFAULT_REGION: \n"); sleep(1)
ENV["AWS_DEFAULT_REGION"] = readline()

print("AWS is now configured.")

In [ ]:
# Run this cell to configure Banyan. When prompted, provide your user ID and API
# key. You can find these on the Account page of your Banyan dashboard.
# If you have already configured Banyan, you can skip this step.

print("Please enter your User ID: \n"); sleep(1)
user_id = readline()
print("Please enter your API Key: \n"); sleep(1)
api_key = readline()

# Configures Banyan client library with your Banyan credentials
configure(user_id=user_id, api_key=api_key)
print("Banyan is now configured.")

## Creating a cluster

For this example, you will need a Banyan cluster. You can either use an existing
cluster or create a new cluster. Run the following code block and enter in either
the name of an existing cluster or the name you would like to use for a new cluster.

If you already have a cluster, you should specify its name, when prompted.

If you would like to instead create a new cluster, provide a name and the name
of the [Amazon EC2 key pair](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html#having-ec2-create-your-key-pair) that you created during [Banyan setup](https://www.banyancomputing.com/creating-clusters).

In the cell below, you can change `instance_type` to create a cluster with a
different EC2 instance type that may have a larger amount of memory or workers.
See the documentation [here](https://www.banyancomputing.com/banyan-jl-docs/create-cluster/) for the other parameters for creating a cluster.

In [ ]:
print("Cluster name for existing cluster or new cluster: \n"); sleep(1)
cluster_name = readline()
println(cluster_name)

clusters = get_clusters()
println("You have $(length(clusters)) clusters")
if !(haskey(clusters, cluster_name) && clusters[cluster_name].status == :running)
    println("Creating new cluster $(cluster_name)")
    print("Name of SSH EC2 Key Pair: \n"); sleep(1)
    ec2_key_pair_name = readline()
    println(ec2_key_pair_name)
    create_cluster(
        name=cluster_name,
        instance_type="t3.2xlarge",
        initial_num_workers=2,
        ec2_key_pair_name=ec2_key_pair_name
    )
else
    println("Using existing cluster $(cluster_name)")
end
get_cluster(cluster_name)

## Running the Simulation

We have a script in `model.jl` which uses MPI (the widely-adopted and battle-tested MPI binding for Julia). This script runs a fluid simulation using the shallow water model. To run this script on a session of multiple workers running in parallel, we call `run_session` and pass in the names of one or more `.jl` files into the `code_files` to run.

In [ ]:
run_session(
    cluster_name = cluster_name, 
    session_name = "shallow water modeling", 
    nworkers = 4, 
    code_files=["file://model.jl"])